Comienzo importando los módulos .py que armé:

In [6]:
import sys
import os
import pprint

#Línea para generar acceso a los módulos
sys.path.append(os.path.join(os.getcwd(), "src"))

from evaluar import calcular_puntaje, mejor_equipo_ronda
from acumular import ranking_innovacion, inicializar_acumulados, actualizar_acumulados, marcar_mejor_equipo
from tabla import tabla_ronda, tabla_resultados_finales, tabla_ranking_innovacion


A continuación, implemento las funciones para resolver la actividad

In [ ]:
evaluaciones = [
 # Ronda 1
 {
 'EquipoA': {'innovacion': 2, 'presentacion': 1, 'errores': True},
 'EquipoB': {'innovacion': 1, 'presentacion': 0, 'errores': False},
 'EquipoC': {'innovacion': 1, 'presentacion': 2, 'errores': True},
 'EquipoD': {'innovacion': 0, 'presentacion': 1, 'errores': False},
 'EquipoE': {'innovacion': 1, 'presentacion': 1, 'errores': False}
 },
 # Ronda 2
 {
 'EquipoA': {'innovacion': 0, 'presentacion': 2, 'errores': False},
 'EquipoB': {'innovacion': 2, 'presentacion': 0, 'errores': True},
 'EquipoC': {'innovacion': 1, 'presentacion': 1, 'errores': False},
 'EquipoD': {'innovacion': 2, 'presentacion': 1, 'errores': True},
 'EquipoE': {'innovacion': 0, 'presentacion': 1, 'errores': False}
 },
 # Ronda 3
 {
 'EquipoA': {'innovacion': 3, 'presentacion': 2, 'errores': False},
 'EquipoB': {'innovacion': 1, 'presentacion': 1, 'errores': True},
 'EquipoC': {'innovacion': 2, 'presentacion': 0, 'errores': False},
 'EquipoD': {'innovacion': 1, 'presentacion': 3, 'errores': True},
 'EquipoE': {'innovacion': 2, 'presentacion': 2, 'errores': False}
 },
 # Ronda 4
 {
 'EquipoA': {'innovacion': 1, 'presentacion': 3, 'errores': True},
 'EquipoB': {'innovacion': 2, 'presentacion': 2, 'errores': False},
 'EquipoC': {'innovacion': 3, 'presentacion': 1, 'errores': False},
 'EquipoD': {'innovacion': 0, 'presentacion': 2, 'errores': True},
 'EquipoE': {'innovacion': 2, 'presentacion': 0, 'errores': False}
 },
 # Ronda 5
 {
 'EquipoA': {'innovacion': 2, 'presentacion': 2, 'errores': False},
 'EquipoB': {'innovacion': 1, 'presentacion': 3, 'errores': True},
 'EquipoC': {'innovacion': 0, 'presentacion': 2, 'errores': False},
 'EquipoD': {'innovacion': 3, 'presentacion': 1, 'errores': False},
 'EquipoE': {'innovacion': 2, 'presentacion': 3, 'errores': True}
 }
]

acumulados = inicializar_acumulados(evaluaciones)        #creo el diccionario de acumulados
for i, ronda in enumerate(evaluaciones, start=1):       #función incrporada que agrega contador a iterable (para imprimir ronda).REEMPLAZA variable contadora i
    dict_ronda = ronda.copy()  # Genero una copia de la ronda para no modificar "evaluaciones"
    mejor_puntaje = -1         # Para ser superado fácilmente
    mejor_nombre = None        # Para ser superado fácilmente
    mayor_total_puntaje = -1
    nro_ronda_mayor_puntaje = None
    
    for nombre_equipo, datos_equipo in ronda.items():
        dict_equipo = datos_equipo.copy()             # Genero copia del equipo para no modificar original
        dict_equipo["puntaje"] = calcular_puntaje(dict_equipo)  # Agrego puntaje en la copia

        # Busco el resultado actualizado de mejor equipo
        mejor_nombre, mejor_puntaje = mejor_equipo_ronda(nombre_equipo, dict_equipo, mejor_puntaje, mejor_nombre)
        
        dict_ronda[nombre_equipo] = dict_equipo     # Agrego copia equipo a copia de ronda

        actualizar_acumulados(acumulados, dict_equipo, nombre_equipo) # Por ahora actualizo sin "mejores"
    

    #Ordeno equipo por puntaje descendente
    equipos_ordenados = sorted(
        dict_ronda.items(),
        key=lambda item: item[1]['puntaje'],                #lambda, FUNCIONES ANÓNIMAS (video)
        reverse=True
    )
    dict_ronda = {
    nombre: {**datos, 'mejores': nombre == mejor_nombre}              #dict comprehension agregandole el dato "mejor", con KWARGS (explicar en video),"podría haber hecho una función y usar map peroes mejor"
    for nombre, datos in equipos_ordenados
    }    

    
    # Agregarle a acumulados lo de mejores por fuera del bucle de equipos, para ahorrar un for
    marcar_mejor_equipo(acumulados, mejor_nombre)

    # Sumar los puntajes de todos los equipos de esta ronda
    total_puntaje_ronda = sum(equipo["puntaje"] for equipo in dict_ronda.values())

    # Si es mayor que el anterior máximo, actualizás
    if total_puntaje_ronda > mayor_total_puntaje:
        mayor_total_puntaje = total_puntaje_ronda
        nro_ronda_mayor_puntaje = i

    # Imprimo la tabla de la ronda
    tabla_ronda(i, dict_ronda)

# Armo el ranking de puntos de innovación
ranking = ranking_innovacion(acumulados)

# Ordenar los acumulados por puntaje descendente
acumulados_ordenados = sorted(
    acumulados.items(),
    key=lambda item: item[1]['puntaje'],
    reverse=True
)

# Imprimo la tabla de los resultados finales:
tabla_resultados_finales(acumulados_ordenados)

# Imprimo ranking de innovación: 
tabla_ranking_innovacion(ranking)
        
print(f"\n Ronda con mayor puntaje total: Ronda {nro_ronda_mayor_puntaje} con {mayor_total_puntaje} puntos.")




 Ronda 1 : 
--------------------------------------------------
Equipo     Innov  Pres   Errores  Puntaje  Mejor 
--------------------------------------------------
EquipoA    2      1      True     6        True  
EquipoC    1      2      True     4        False 
EquipoE    1      1      False    4        False 
EquipoB    1      0      False    3        False 
EquipoD    0      1      False    1        False 

 Ronda 2 : 
--------------------------------------------------
Equipo     Innov  Pres   Errores  Puntaje  Mejor 
--------------------------------------------------
EquipoD    2      1      True     6        True  
EquipoB    2      0      True     5        False 
EquipoC    1      1      False    4        False 
EquipoA    0      2      False    2        False 
EquipoE    0      1      False    1        False 

 Ronda 3 : 
--------------------------------------------------
Equipo     Innov  Pres   Errores  Puntaje  Mejor 
--------------------------------------------------
Equip